In [3]:
import os.path
import logging
import torch.utils.data
import numpy as np

from dataset.v2x_utils import Filter, RectFilter
from dataset.base_dataset import build_path_to_info, get_annos
from dataset.dataset_utils import load_json, InfFrame, VehFrame, VICFrame, Label

In [8]:
np.random.rand(3, 2, 2).squeeze(axis=-1).shape

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [6]:
class VICDataset(torch.utils.data.Dataset):
    def __init__(self, path, split="train", sensortype="lidar", extended_range=None, val_data_path=""):
        super().__init__()
        self.path = path
        self.inf_path2info = build_path_to_info(
            "infrastructure-side",
            load_json(os.path.join(path, "infrastructure-side/data_info.json")),
            sensortype,
        )
        self.veh_path2info = build_path_to_info(
            "vehicle-side",
            load_json(os.path.join(path, "vehicle-side/data_info.json")),
            sensortype,
        )

        ### Patch for FFNet evaluation ###
        # if args.model =='feature_flow':
        #     frame_pairs = load_json(val_data_path)
        # else:
        #     frame_pairs = load_json(os.path.join(path, "cooperative/data_info.json"))
        #     split_path = args.split_data_path
        #     frame_pairs = self.get_split(split_path, split, frame_pairs)
        frame_pairs = load_json(val_data_path)

        self.data = []
        self.inf_frames = {}
        self.veh_frames = {}

        for elem in frame_pairs:
            if sensortype == "lidar":
                inf_frame = self.inf_path2info[elem["infrastructure_pointcloud_path"]]
                veh_frame = self.veh_path2info[elem["vehicle_pointcloud_path"]]
            elif sensortype == "camera":
                inf_frame = self.inf_path2info[elem["infrastructure_image_path"]]
                veh_frame = self.veh_path2info[elem["vehicle_image_path"]]
                get_annos(path, "infrastructure-side", inf_frame, "camera")
                get_annos(path, "vehicle-side", veh_frame, "camera")

            inf_frame = InfFrame(path + "/infrastructure-side/", inf_frame)
            veh_frame = VehFrame(path + "/vehicle-side/", veh_frame)
            if not inf_frame["batch_id"] in self.inf_frames:
                self.inf_frames[inf_frame["batch_id"]] = [inf_frame]
            else:
                self.inf_frames[inf_frame["batch_id"]].append(inf_frame)
            if not veh_frame["batch_id"] in self.veh_frames:
                self.veh_frames[veh_frame["batch_id"]] = [veh_frame]
            else:
                self.veh_frames[veh_frame["batch_id"]].append(veh_frame)
            vic_frame = VICFrame(path, elem, veh_frame, inf_frame, 0)

            # filter in world coordinate
            if extended_range is not None:
                trans = vic_frame.transform(from_coord="Vehicle_lidar", to_coord="World")
                filt_world = RectFilter(trans(extended_range)[0])

            trans_1 = vic_frame.transform("World", "Vehicle_lidar")
            label_v = Label(os.path.join(path, elem["cooperative_label_path"]), filt_world)
            label_v["boxes_3d"] = trans_1(label_v["boxes_3d"])
            filt = RectFilter(extended_range[0])
            tup = (
                vic_frame,
                label_v,
                filt,
            )
            self.data.append(tup)

    def query_veh_segment(self, frame, sensortype="lidar", previous_only=False):
        segment = self.veh_frames[frame.batch_id]
        return [f for f in segment if f.id[sensortype] < frame.id[sensortype] or not previous_only]

    def query_inf_segment(self, frame, sensortype="lidar", previous_only=False):
        segment = self.inf_frames[frame.batch_id]
        return [f for f in segment if f.id[sensortype] < frame.id[sensortype] or not previous_only]

    def get_split(self, split_path, split, frame_pairs):
        if os.path.exists(split_path):
            split_data = load_json(split_path)
        else:
            print("Split File Doesn't Exists!")
            raise Exception

        if split in ["train", "val", "test"]:
            split_data = split_data["cooperative_split"][split]
        else:
            print("Split Method Doesn't Exists!")
            raise Exception

        frame_pairs_split = []
        for frame_pair in frame_pairs:
            veh_frame_idx = frame_pair["vehicle_image_path"].split("/")[-1].replace(".jpg", "")
            if veh_frame_idx in split_data:
                frame_pairs_split.append(frame_pair)
        return frame_pairs_split

    def __getitem__(self, index):
        raise NotImplementedError


class VICSyncDataset(VICDataset):
    def __init__(self, path, split="train", sensortype="lidar", extended_range=None, val_data_path=""):
        super().__init__(path, split, sensortype, extended_range, val_data_path)
        logging.info("VIC-Sync {} dataset, overall {} frames".format(split, len(self.data)))

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [8]:
VICSyncDataset(
    path='/Volumes/BetaCat-USB/DAIR-V2X-C/cooperative-vehicle-infrastructure',
    split='train',
    sensortype='camera',
)

FileNotFoundError: [Errno 2] No such file or directory: ''